#### Python script for reading and cleaning airport, route, and airline datasets from [OpenFlights](https://openflights.org/data.html) to be read and processed in C++.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Get relevant data for airports in the U.S.
colnames = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Olson Timezone', 'Type', 'Source']
airport_df = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', names=colnames)
airport_df = airport_df[airport_df['Country'] == 'United States']
airport_df.drop(['Country', 'IATA', 'ICAO', 'Altitude', 'Timezone', 'DST', 'Olson Timezone', 'Type', 'Source', 'Latitude', 'Longitude'], axis=1, inplace=True)

In [3]:
# Get relevant data for airlines.
colnames = ['Airline ID', 'Airline', 'Alias', 'IATA', 'ICAO', 'Callsign', 'Country', 'Active']
airline_df = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat', names=colnames)
airline_df.drop(['Alias', 'IATA', 'ICAO', 'Callsign', 'Country', 'Active'], axis=1, inplace=True)
airline_df.head()

,Airline ID,Airline
0,-1,Unknown
1,1,Private flight
2,2,135 Airways
3,3,1Time Airline
4,4,2 Sqn No 1 Elementary Flying Training School


In [4]:
# Get relevant data for routes.
colnames = ['Airline', 'Airline ID', 'Source Airport', 'Source Airport ID', 'Destination Airport', 'Destination Airport ID', 'Codeshare', 'Stops', 'Equipment']
route_df = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', names=colnames)
route_df.drop(['Airline', 'Source Airport', 'Destination Airport', 'Codeshare', 'Stops', 'Equipment'], axis=1, inplace=True)

In [5]:
# Remove routes with missing data.
route_df.replace('\\N', np.NaN, inplace=True)
route_df.dropna(how='any', inplace=True)

In [6]:
# Typecast route data for comparison.
route_df = route_df.astype(int)

In [7]:
# Get routes with source and destination airports in the U.S.
airports = airport_df['Airport ID'].values
route_df = route_df.loc[route_df['Source Airport ID'].isin(airports) & route_df['Destination Airport ID'].isin(airports)]
route_df.reset_index(drop=True, inplace=True)

In [8]:
# Remove airports with no routes.
srcs = route_df['Source Airport ID'].values
dests = route_df['Destination Airport ID'].values
airport_df = airport_df.loc[airport_df['Airport ID'].isin(srcs) | airport_df['Airport ID'].isin(dests)]
airport_df.reset_index(drop=True, inplace=True)

In [9]:
# Create dataframe to map airport ids.
map_df = pd.DataFrame()
map_df['Old ID'] = airport_df['Airport ID']
map_df['New ID'] = range(0, map_df.shape[0])
map_df.head()

,Old ID,New ID
0,3411,0
1,3413,1
2,3414,2
3,3415,3
4,3417,4


In [10]:
# No longer need airport ID in airports.
airport_df.drop(['Airport ID'], axis=1, inplace=True)
airport_df.head()

,Name,City
0,Barter Island LRRS Airport,Barter Island
1,Cape Lisburne LRRS Airport,Cape Lisburne
2,Point Lay LRRS Airport,Point Lay
3,Hilo International Airport,Hilo
4,Bettles Airport,Bettles


In [11]:
# Map airport ids in routes to index in airports.
route_df = route_df.replace(map_df.set_index('Old ID')['New ID'])

In [12]:
# Merge airline data with route data.
route_df = route_df.merge(airline_df)
route_df.drop(['Airline ID'], axis=1, inplace=True)
route_df.head()

,Source Airport ID,Destination Airport ID,Airline
0,51,498,Air Salone
1,498,497,Air Salone
2,347,200,Air Choice One
3,347,122,Air Choice One
4,267,200,Air Choice One


In [13]:
# Generate airline weights.
airline_weights = route_df['Airline'].value_counts()

In [14]:
# Normalize airline weights.
airline_weights = np.average(airline_weights) / airline_weights

In [15]:
# Convert to dataframe.
airline_weights = pd.DataFrame(airline_weights).reset_index()
airline_weights.columns = ['Airline', 'Weight']
airline_weights.head()

,Airline,Weight
0,American Airlines,0.102978
1,US Airways,0.109730
2,Delta Air Lines,0.109730
3,United Airlines,0.111751
4,Southwest Airlines,0.129317


In [16]:
# Generate airport weights.
airports = pd.concat([route_df['Source Airport ID'], route_df['Destination Airport ID']])
airport_weights = airports.value_counts()

In [17]:
# Normalize airport weights.
airport_weights = np.average(airport_weights) / airport_weights

In [18]:
# Convert to dataframe.
airport_weights = pd.DataFrame(airport_weights).reset_index()
airport_weights.columns = ['Destination Airport ID', 'Airport Weight']
airport_weights.head()

,Destination Airport ID,Airport Weight
0,124,0.026004
1,200,0.051731
2,116,0.059211
3,168,0.059574
4,31,0.064194


In [19]:
# Merge weights with route data.
route_df = route_df.merge(airline_weights)
route_df = route_df.merge(airport_weights)
route_df['Weight'] = route_df['Weight'] + route_df['Airport Weight']
route_df.drop(['Airport Weight'], axis=1, inplace=True)
route_df.head()

,Source Airport ID,Destination Airport ID,Airline,Weight
0,51,498,Air Salone,92.256390
1,498,497,Air Salone,111.707225
2,347,200,Air Choice One,14.612842
3,267,200,Air Choice One,14.612842
4,252,200,American Airlines,0.154709


In [20]:
# Save airport & route data as csv files.
airport_df.to_csv('airports.csv', index=False, header=False, line_terminator=',\n')
route_df.to_csv('routes.csv', index=False, header=False, line_terminator=',\n')

In [21]:
# Total statistics:
print("Total Airports:", airport_df.shape[0])
print("Total Routes:", route_df.shape[0])
print("Total Airlines:", len(route_df['Airline'].unique()))
print("Total Cities:", len(airport_df['City'].unique()))

print("\nSource Airports:", len(route_df['Source Airport ID'].unique()))
print("Destination Airports:", len(route_df['Destination Airport ID'].unique()))

Total Airports: 539
Total Routes: 10484
Total Airlines: 72
Total Cities: 519

Source Airports: 533
Destination Airports: 532


In [22]:
# Top airport statistics:
top_airports = route_df.groupby(['Source Airport ID']).count().sort_values(by='Weight', ascending=False)
top_airports.head()

,Destination Airport ID,Airline,Weight
Source Airport ID,,,
124,755,755,755
200,380,380,380
116,330,330,330
168,320,320,320
31,297,297,297


In [23]:
# Top airline statistics:
top_airlines = route_df.groupby(['Airline']).count().sort_values(by='Weight', ascending=False)
top_airlines.head()

,Source Airport ID,Destination Airport ID,Weight
Airline,,,
American Airlines,1414,1414,1414
Delta Air Lines,1327,1327,1327
US Airways,1327,1327,1327
United Airlines,1303,1303,1303
Southwest Airlines,1126,1126,1126


In [24]:

print("Median Routes/Airport: ", top_airports.median()[0])
print("Mean Routes/Airport: ", top_airports.mean()[0])

print("\nMedian Routes/Airline: ", top_airlines.median()[0])
print("Mean Routes/Airline: ", top_airlines.mean()[0])

Median Routes/Airport:  4.0
Mean Routes/Airport:  19.669793621013135

Median Routes/Airline:  11.5
Mean Routes/Airline:  145.61111111111111


In [25]:
print("Median Cost: ", route_df['Weight'].median())
print("Average Cost: ", route_df['Weight'].mean())
print("Total Graph Cost: ", route_df['Weight'].sum())

Median Cost:  0.4590869980003377
Average Cost:  2.0007158540814274
Total Graph Cost:  20975.505014189686
